In [3]:
from pathlib import Path
import pandas as pd

# Data overview

https://www.kaggle.com/competitions/mercari-price-suggestion-challenge/data

## Loading

In [6]:
data = Path('../data')

In [9]:
df = pd.read_table(data/'train.tsv')
df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [10]:
df.shape

(1482535, 8)

## Categories

In [13]:
categorical = []

### item_condition_id

The condition of the items provided by the seller

In [14]:
df.item_condition_id.value_counts()

1    640549
3    432161
2    375479
4     31962
5      2384
Name: item_condition_id, dtype: int64

In [15]:
categorical.append('item_condition_id')

### category_name

Category of the listing.

In [16]:
df.category_name.value_counts()

Women/Athletic Apparel/Pants, Tights, Leggings    60177
Women/Tops & Blouses/T-Shirts                     46380
Beauty/Makeup/Face                                34335
Beauty/Makeup/Lips                                29910
Electronics/Video Games & Consoles/Games          26557
                                                  ...  
Handmade/Dolls and Miniatures/Primitive               1
Handmade/Pets/Tag                                     1
Handmade/Patterns/Accessories                         1
Home/Kids' Home Store/Nursery Furniture               1
Handmade/Pets/Blanket                                 1
Name: category_name, Length: 1287, dtype: int64

There seems to be a structure to the categories. Maybe later they can be split into levels?

There are some missing categories:

In [22]:
df.category_name.isna().sum()

6327

For now we won't treat them differently, as fastai handles missing values.

In [24]:
categorical.append('category_name')